In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
# from keras.models import Sequential, Model
import keras.models
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

## Engineering Features

In [29]:
# select features 'mel', 'mfcc' or 'freq_power'
def selectFeatures(feature='mel'):
    feat = os.path.join("bird_data","features")
    train_folder = os.path.join(feat,f'{feature}_features', 'train')
    test_folder = os.path.join(feat,f'{feature}_features', 'test')
    val_folder = os.path.join(feat,f'{feature}_features', 'val')
    return train_folder, test_folder, val_folder
    

In [30]:
train_folder, test_folder, val_folder = selectFeatures('mel')

# Neural Network

In [32]:
# Batch and target size
BATCH_SIZE = 16
TARGET_SIZE = (720,288)


In [33]:
# model
model = keras.models.Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(720, 288, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 718, 286, 32)      896       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 716, 284, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 358, 142, 64)      0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 358, 142, 64)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3253504)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                208224320 
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)               

In [34]:
early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
callbacks = [early_stopping]

In [35]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,  
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.1,
                                   fill_mode='nearest',
                                   horizontal_flip = False)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [36]:
test_set = test_datagen.flow_from_directory(test_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

Found 200 images belonging to 10 classes.


In [37]:
training_set = train_datagen.flow_from_directory(train_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical', shuffle=True)

Found 700 images belonging to 10 classes.


In [40]:
val_set = test_datagen.flow_from_directory(val_folder, target_size = TARGET_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')

Found 100 images belonging to 10 classes.


In [41]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(training_set.classes), training_set.classes)
weights = {label:w for label, w in zip(np.unique(training_set.classes), class_weights)}

In [ ]:
model.fit(training_set, epochs=10, class_weight=weights,  validation_data = test_set, verbose=True, callbacks=callbacks, shuffle=True)

In [ ]:
Y_pred = model.predict_generator(val_set)
y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(val_set.classes, y_pred))

In [ ]:
score = model.evaluate(training_set, verbose=0)
print("Training Accuracy: ", score[1])

In [ ]:
score = model.evaluate(test_set, verbose=0)
print("Testing Accuracy: ", score[1])

In [ ]:
score = model.evaluate_generator(val_set, verbose=0)
print("Validating Accuracy: ", score[1])